In [1]:
import pyspark
from pyspark.sql import types
from pyspark.sql import functions as F
import pandas as pd

In [2]:
from pyspark.sql import SparkSession

In [3]:
pyspark.__file__

'/root/spark/spark-3.0.3-bin-hadoop3.2/python/pyspark/__init__.py'

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/26 13:14:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/26 13:14:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/26 13:14:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/02/26 13:14:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/02/26 13:14:09 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


## 1

In [9]:
spark.version

'3.0.3'

Answer 1: 3.0.3

## 2

In [10]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-26 09:46:20--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.64.20
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.64.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  36.1MB/s    in 21s     

2022-02-26 09:46:41 (34.1 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [5]:
df = spark.read \
     .option("header", "true") \
     .csv('fhvhv_tripdata_2021-02.csv')

In [12]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [6]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [7]:
df = spark.read \
     .option("header", "true") \
     .schema(schema) \
     .csv('fhvhv_tripdata_2021-02.csv')

In [8]:
df = df.repartition(24)

In [17]:
df.write.parquet("fhvhv/2021/02", mode="overwrite")

In [19]:
!ls -lh fhvhv/2021/02

total 208M
-rw-r--r-- 1 root root    0 Feb 26 10:00 _SUCCESS
-rw-r--r-- 1 root root 8.7M Feb 26 10:00 part-00000-2ecf7a4c-3788-4a2a-bce9-388475ada610-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 26 10:00 part-00001-2ecf7a4c-3788-4a2a-bce9-388475ada610-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 26 10:00 part-00002-2ecf7a4c-3788-4a2a-bce9-388475ada610-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 26 10:00 part-00003-2ecf7a4c-3788-4a2a-bce9-388475ada610-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 26 10:00 part-00004-2ecf7a4c-3788-4a2a-bce9-388475ada610-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 26 10:00 part-00005-2ecf7a4c-3788-4a2a-bce9-388475ada610-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 26 10:00 part-00006-2ecf7a4c-3788-4a2a-bce9-388475ada610-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 26 10:00 part-00007-2ecf7a4c-3788-4a2a-bce9-388475ada610-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 26 10:00 part-00008-2ecf7a4c-3788-4

Answer 2: 208MB

## 3

In [23]:
df.select('hvfhs_license_num') \
    .filter(F.to_date(df.pickup_datetime) == '2021-02-15') \
    .count()

367170

Answer 3: 367170

## 4

In [24]:
df = df.withColumn("duration", (F.unix_timestamp(df.dropoff_datetime) - F.unix_timestamp(df.pickup_datetime)))

In [25]:
df.select('duration','pickup_datetime') \
    .sort(F.desc('duration')) \
    .show(5)

+--------+-------------------+
|duration|    pickup_datetime|
+--------+-------------------+
|   75540|2021-02-11 13:40:44|
|   57221|2021-02-17 15:54:53|
|   44039|2021-02-20 12:08:15|
|   40653|2021-02-03 20:24:25|
|   37577|2021-02-19 23:17:44|
+--------+-------------------+
only showing top 5 rows



Answer 4: 2021-02-11

## 5

In [9]:
df.registerTempTable('fhvhv_tripdata')

query = spark.sql("""
select dispatching_base_num, count(dispatching_base_num)
from fhvhv_tripdata
group by 1
order by 2 desc
""")

In [10]:
query.show()

+--------------------+---------------------------+
|dispatching_base_num|count(dispatching_base_num)|
+--------------------+---------------------------+
|              B02510|                    3233664|
|              B02764|                     965568|
|              B02872|                     882689|
|              B02875|                     685390|
|              B02765|                     559768|
|              B02869|                     429720|
|              B02887|                     322331|
|              B02871|                     312364|
|              B02864|                     311603|
|              B02866|                     311089|
|              B02878|                     305185|
|              B02682|                     303255|
|              B02617|                     274510|
|              B02883|                     251617|
|              B02884|                     244963|
|              B02882|                     232173|
|              B02876|         

Answer 5: 3

## 6

In [12]:
zones = spark.read.csv("taxi+_zone_lookup.csv", header=True)
zones.registerTempTable('zones')

In [14]:
zones.head(10)

[Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(LocationID='2', Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone'),
 Row(LocationID='3', Borough='Bronx', Zone='Allerton/Pelham Gardens', service_zone='Boro Zone'),
 Row(LocationID='4', Borough='Manhattan', Zone='Alphabet City', service_zone='Yellow Zone'),
 Row(LocationID='5', Borough='Staten Island', Zone='Arden Heights', service_zone='Boro Zone'),
 Row(LocationID='6', Borough='Staten Island', Zone='Arrochar/Fort Wadsworth', service_zone='Boro Zone'),
 Row(LocationID='7', Borough='Queens', Zone='Astoria', service_zone='Boro Zone'),
 Row(LocationID='8', Borough='Queens', Zone='Astoria Park', service_zone='Boro Zone'),
 Row(LocationID='9', Borough='Queens', Zone='Auburndale', service_zone='Boro Zone'),
 Row(LocationID='10', Borough='Queens', Zone='Baisley Park', service_zone='Boro Zone')]

In [18]:
query = spark.sql("""
select zones_pu.Zone, zones_do.Zone, count(dispatching_base_num)
from fhvhv_tripdata
left join zones as zones_pu on fhvhv_tripdata.PUlocationid = zones_pu.Locationid
left join zones as zones_do on fhvhv_tripdata.DOlocationid = zones_do.Locationid
where fhvhv_tripdata.PUlocationid != fhvhv_tripdata.DOlocationid
group by 1, 2
order by 3 desc
limit 5
""")

In [19]:
query.show()

+--------------------+--------------------+---------------------------+
|                Zone|                Zone|count(dispatching_base_num)|
+--------------------+--------------------+---------------------------+
|         JFK Airport|                  NA|                      12542|
|Prospect-Lefferts...| Crown Heights North|                      11814|
|      Bushwick North|      Bushwick South|                      11491|
|      Bushwick South|      Bushwick North|                      11487|
| Crown Heights North|Prospect-Lefferts...|                      11462|
+--------------------+--------------------+---------------------------+



Answer 6: JFK Airport / NA